<a href="https://colab.research.google.com/github/UFResearchComputing/gatorAI_summer_camp_2024/blob/main/01_full_of_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><img src="images/gator_ai_camp_2024_logo_200.png" align="right">

# Gator AI Summer Camp 2025

In this notebook, we're going to use Python to create a deep learning model that can take images of faces and output the emotion being expressed.

The dataset we're going to use is the FER-2013 dataset, which contains 35,887 grayscale images of faces. Each image is 48x48 pixels and is labeled with one of seven emotions: anger, disgust, fear, happiness, sadness, surprise, or neutral. The dataset and more information can be found [on Kaggle](https://www.kaggle.com/datasets/msambare/fer2013/data).

**Note:** One issue with the dataset is that it has relatively few images in the disgust category, so we drop that category for this exercise.

To build our model, we'll use the Keras deep learning library, which provides a high-level interface for building and training neural networks. We'll start by loading the dataset and exploring the images, then we'll build and train a convolutional neural network (CNN) to classify the emotions in the images.

**Before you get started, make sure to select a Runtime with a GPU!** <img src='images/colab_change_runtime_type.png' align='right' width='50%' alt='Image of the Runtime menu options in Google Colab'>
* Go to the **"Runtime"** menu
* Select **"Change runtime type"**
* Select **"T4 GPU"** and click **"Save"**

In [1]:
# Import the necessary libraries
import os
import sys
import shutil
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from functools import reduce
import itertools
from collections import Counter
import kagglehub


# Import the libraries for CNNs
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

# Import the libraries for the evaluation
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
# Download the dataset using kagglehub
print("Downloading dataset from Kaggle...")
dataset_path = kagglehub.dataset_download("msambare/fer2013")
print(f"Dataset downloaded to: {dataset_path}")

# Create data directory if it doesn't exist
if not os.path.exists("data"):
    os.makedirs("data")
    print("Created 'data' directory")

# Check what files/folders are in the downloaded dataset
print(f"Contents of {dataset_path}:")
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        print(f"  Directory: {item}")
    else:
        print(f"  File: {item}")

# Look for zip file first
zip_file = None
for file in os.listdir(dataset_path):
    if file.endswith(".zip"):
        zip_file = os.path.join(dataset_path, file)
        break

if zip_file:
    # Extract the zip file to the data directory
    print(f"Extracting {zip_file} to data/")
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall("data/")
    print("Extraction complete")
else:
    # No zip file found, check if train/test directories already exist
    train_dir = os.path.join(dataset_path, "train")
    test_dir = os.path.join(dataset_path, "test")

    if os.path.exists(train_dir) and os.path.exists(test_dir):
        print("Found train and test directories, copying to data/")
        # Copy the train and test directories
        shutil.copytree(train_dir, os.path.join("data", "train"), dirs_exist_ok=True)
        shutil.copytree(test_dir, os.path.join("data", "test"), dirs_exist_ok=True)
        print("Directories copied successfully")
    else:
        # Look for any other structure
        print("Searching for dataset files in subdirectories...")
        for root, dirs, files in os.walk(dataset_path):
            if "train" in dirs and "test" in dirs:
                train_source = os.path.join(root, "train")
                test_source = os.path.join(root, "test")
                print(f"Found train/test directories in: {root}")
                shutil.copytree(
                    train_source, os.path.join("data", "train"), dirs_exist_ok=True
                )
                shutil.copytree(
                    test_source, os.path.join("data", "test"), dirs_exist_ok=True
                )
                print("Directories copied successfully")
                break
        else:
            print("Could not find train/test directories in the downloaded dataset")

# Verify the data directory structure
if os.path.exists("data"):
    print(f"\nContents of data directory:")
    for item in os.listdir("data"):
        item_path = os.path.join("data", item)
        if os.path.isdir(item_path):
            print(f"  Directory: {item}")
            # Show subdirectories (emotion categories)
            if os.path.exists(item_path):
                subdirs = [
                    d
                    for d in os.listdir(item_path)
                    if os.path.isdir(os.path.join(item_path, d))
                ]
                print(f"    Emotion categories: {subdirs}")

# Delete the disgust folders as there are so few examples in that category
disgust_train_path = os.path.join("data", "train", "disgust")
disgust_test_path = os.path.join("data", "test", "disgust")

if os.path.exists(disgust_train_path):
    shutil.rmtree(disgust_train_path)
    print("Removed data/train/disgust folder")

if os.path.exists(disgust_test_path):
    shutil.rmtree(disgust_test_path)
    print("Removed data/test/disgust folder")

print("Dataset preparation complete!")

Dataset downloaded to: C:\Users\i.lutticken\.cache\kagglehub\datasets\msambare\fer2013\versions\1
Contents of C:\Users\i.lutticken\.cache\kagglehub\datasets\msambare\fer2013\versions\1:
  Directory: test
  Directory: train
Found train and test directories, copying to data/
Directories copied successfully

Contents of data directory:
  Directory: test
    Emotion categories: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
  Directory: train
    Emotion categories: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Removed data/train/disgust folder
Removed data/test/disgust folder
Dataset preparation complete!


In [4]:
def load_display_data(
    path,
    batch_size=32,
    shape=(80, 80, 1),
    show_pictures=True,
    show_histogram=True,
    augment=False,
    balance_classes=False,
):
    """Takes a path, batch size, target shape for images and optionally whether to show sample images, augment data, and balance classes.


    Returns training and testing datasets


    """


    print("***********************************************************************")


    print("Load data:")


    print(f"  - Loading the dataset from: {path}.")


    print(f"  - Using a batch size of: {batch_size}.")


    print(f"  - Resizing input images to: {shape}.")


    print(f"  - Data augmentation: {augment}.")


    print(f"  - Balance classes: {balance_classes}.")


    print("***********************************************************************")


    # Define the directory path


    directory_path = path


    # Define the batch size


    batch_size = batch_size


    # Define the image size using the 1st 2 elements of the shape parameter


    # We don't need the number of channels here, just the dimensions to use


    image_size = shape[:2]


    # Create the data augmentation pipeline if augmentation is enabled


    data_augmentation = (
        tf.keras.Sequential(
            [
                tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
                tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
                tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
                tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
            ]
        )
        if augment
        else None
    )


    # Load the dataset


    X_train = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(directory_path, "train"),
        batch_size=batch_size,
        image_size=image_size,
        labels="inferred",
        label_mode="int",
        color_mode="grayscale",
    )


    X_val = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(directory_path, "test"),
        batch_size=batch_size,
        image_size=image_size,
        labels="inferred",
        label_mode="int",
        color_mode="grayscale",
    )


    # Store class names before transforming the dataset


    class_names = X_train.class_names


    # Apply data augmentation to the training dataset if enabled


    if augment:


        X_train = X_train.map(lambda x, y: (data_augmentation(x, training=True), y))


    if show_pictures:
        print(class_names)


        # Display up to 3 images from each of the categories


        class_images_dict = {class_name: [] for class_name in class_names}


        # Collect images for each class


        for images, labels in X_train:


            images = images.numpy()


            labels = labels.numpy()


            for i, class_name in enumerate(class_names):


                if len(class_images_dict[class_name]) < 3:


                    # Filter images of the current class


                    class_images = images[labels == i]


                    class_images_dict[class_name].extend(
                        class_images[: 3 - len(class_images_dict[class_name])]
                    )


            # Break if we have collected enough images for all classes


            if all(len(imgs) >= 3 for imgs in class_images_dict.values()):


                break


        # Display the collected images


        for class_name, class_images in class_images_dict.items():


            fig, axs = plt.subplots(1, 3, figsize=(10, 10))


            fig.subplots_adjust(
                wspace=0, hspace=30
            )  # Adjust the space between subplots


            for j in range(len(class_images)):


                ax = axs[j]


                ax.imshow(class_images[j].astype("uint8"), cmap="gray")


                ax.set_title(class_name)


                ax.axis("off")


                ax.set_xticklabels([])


                ax.set_yticklabels([])


                ax.set_aspect("equal")


            plt.show()


    train_labels = []


    if show_histogram:


        # Collect all labels for training and validation datasets


        for images, labels in X_train:


            train_labels.extend(labels.numpy())


        val_labels = []


        for images, labels in X_val:


            val_labels.extend(labels.numpy())


        # Display the class distribution for the entire dataset


        plt.figure(figsize=(15, 5))


        plt.subplot(1, 3, 1)


        plt.title("Dataset")


        # Title for the total dataset


        plt.hist(
            train_labels + val_labels,
            bins=np.arange(len(class_names) + 1) - 0.5,
            edgecolor="black",
        )


        plt.xticks(range(len(class_names)), labels=class_names, rotation=45)


        plt.subplot(1, 3, 2)


        plt.title("Training")


        plt.hist(
            train_labels, bins=np.arange(len(class_names) + 1) - 0.5, edgecolor="black"
        )


        plt.xticks(range(len(class_names)), labels=class_names, rotation=45)


        plt.subplot(1, 3, 3)


        plt.title("Validation")


        plt.hist(
            val_labels, bins=np.arange(len(class_names) + 1) - 0.5, edgecolor="black"
        )


        plt.xticks(range(len(class_names)), labels=class_names, rotation=45)


        plt.tight_layout()


        plt.show()


    if balance_classes:


        class_weights = class_weight.compute_class_weight(
            class_weight="balanced", classes=np.unique(train_labels), y=train_labels
        )


        class_weights = tf.constant(
            list(class_weights)
        )  # Convert to TensorFlow constant


        # Apply class weights to the training dataset using tf.gather


        X_train = X_train.map(
            lambda img, label: (
                img,
                label,
                tf.gather(class_weights, tf.cast(label, tf.int32)),
            )
        )


        # (If needed, you could also adjust the validation set weights, but typically not necessary)


        # X_val = X_val.map(lambda img, label: (img, label, tf.gather(class_weights, tf.cast(label, tf.int32))))


    return X_train, X_val



data_path = "data/"



X_train, X_val = load_display_data(
    data_path,
    batch_size=32,
    shape=(80, 80, 1),
    show_pictures=True,
    show_histogram=True,
    augment=True,
    balance_classes=True,
)

***********************************************************************
Load data:
  - Loading the dataset from: data/.
  - Using a batch size of: 32.
  - Resizing input images to: (80, 80, 1).
  - Data augmentation: True.
  - Balance classes: True.
***********************************************************************


AttributeError: module 'tensorflow.keras.layers' has no attribute 'experimental'

In [ ]:
# Define the CNN model architecture


def create_model(
    input_shape, num_classes, padding="same", activation="relu", dropout_rate=0.2
):
    """Takes the input shape and number of classes and returns a CNN model"""
    print("***********************************************************************")
    print("Create model:")
    print(f"  - Input shape: {input_shape}.")
    print(f"  - Number of classes: {num_classes}.")
    print("***********************************************************************")
    # Create the model
    model = Sequential(
        [
            Conv2D(
                16, 3, padding=padding, activation=activation, input_shape=input_shape
            ),
            MaxPooling2D(),
            Dropout(dropout_rate),
            Conv2D(32, 3, padding=padding, activation=activation),
            MaxPooling2D(),
            Dropout(dropout_rate),
            Conv2D(64, 3, padding=padding, activation=activation),
            MaxPooling2D(),
            Dropout(dropout_rate),
            Flatten(),
            Dense(128, activation=activation),
            Dense(num_classes, activation="softmax"),
        ]
    )
    # Visualize the model
    model.summary()

    return model


# Define our instance of the model

input_shape = (80, 80, 1)
num_classes = 6

model = create_model(input_shape, num_classes, padding="same", activation="relu")

In [ ]:
# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

early_stop = EarlyStopping(monitor="val_loss", patience=3, verbose=1, mode="min")

In [ ]:
# Train the model
history = model.fit(X_train, validation_data=X_val, epochs=30, callbacks=[early_stop])

In [ ]:
# Evaluate the model
print("***********************************************************************")
print("Accuracy and Loss per Epoch:")
print("***********************************************************************")
loss, accuracy = model.evaluate(X_val)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Plot the training history
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0, 1])
plt.legend(loc="lower right")
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.title("Loss")
plt.show()


# Define a function to plot the confusion matrix
def plot_confusion_matrix(model, dataset):
    print("***********************************************************************")
    print("Confusion matrix:")
    print("***********************************************************************")

    # Get the true labels
    y_true = []
    for images, labels in dataset:
        y_true.extend(labels.numpy())

    # Get the predicted labels
    y_pred = model.predict(dataset)
    y_pred = np.argmax(y_pred, axis=1)

    # Get the class names
    class_names = dataset.class_names

    # Calculate the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot the confusion matrix
    plt.figure(figsize=(15, 15))
    plt.imshow(cm, cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.xticks(range(len(class_names)), labels=class_names, rotation=45)
    plt.yticks(range(len(class_names)), labels=class_names)

    # Annotate each cell with the numeric value
    thresh = cm.max() / 2
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(
                j,
                i,
                format(cm[i, j], "d"),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
            )

    plt.show()


# Example usage (assuming X_val is your validation dataset)
plot_confusion_matrix(model, X_val)

## Save our model

Now that we've trained our model, we can save it for the next steps where we will want to use the model.


In [ ]:
model.save("emotion_model.keras")